In [15]:

!pip install transformers sentencepiece --quiet


In [16]:

from transformers import pipeline
import textwrap
from IPython.display import display, Markdown


In [17]:

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
headline_gen = pipeline("summarization", model="google/pegasus-xsum")


Device set to use cpu


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cpu


In [18]:


def split_into_chunks(text, max_chunk_size=1000):
    """Split long text into chunks suitable for summarization."""
    return [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]

def summarize_article(text):
    """Generate a paragraph summary of the article."""
    chunks = split_into_chunks(text)
    summary = ""
    for chunk in chunks:
        out = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
        summary += out[0]['summary_text'] + " "
    return summary.strip()

def generate_headline(text):
    """Generate a short headline."""
    result = headline_gen(text, max_length=15, min_length=5, do_sample=False)
    return result[0]['summary_text']

def extract_subpoints(summary):
    """Extract 2-5 bullet points from the summary."""
    sentences = summary.split('. ')
    return [f"• {s.strip()}." for s in sentences if s][:5]


In [42]:
# 📌 STEP 5: Input News Article (you can paste your article here)
article = """
The European Union is preparing to launch a new set of climate regulations aimed at significantly reducing carbon emissions by 2030. The proposal, which is expected to be unveiled this week, includes plans for stricter vehicle emission standards, increased investment in renewable energy, and the implementation of a carbon border tax. The initiative is part of the EU’s broader Green Deal strategy, which seeks to make the continent climate-neutral by 2050. Officials say the changes could reshape the region’s economy, affecting industries ranging from automotive manufacturing to agriculture. Environmental groups have largely welcomed the move, though some businesses have expressed concern about the potential costs of compliance.
"""


In [20]:

headline = generate_headline(article)
summary = summarize_article(article)
subpoints = extract_subpoints(summary)


In [21]:


print("\n📰 Headline:")
display(Markdown(f"**{headline}**"))

print("\n📌 Key Subpoints:")
for point in subpoints:
    display(Markdown(point))

print("\n📄 Summary Paragraph:")
wrapped_summary = textwrap.fill(summary, width=100)
display(Markdown(f"{wrapped_summary}"))



📰 Headline:


**Images courtesy of AFP, EPA, Getty Images and Reuters**


📌 Key Subpoints:


• Proposal includes stricter vehicle emission standards, increased investment in renewable energy.

• The initiative is part of the EU’s broader Green Deal strategy, which seeks to make the continent climate-neutral..


📄 Summary Paragraph:


Proposal includes stricter vehicle emission standards, increased investment in renewable energy. The
initiative is part of the EU’s broader Green Deal strategy, which seeks to make the continent
climate-neutral.

In [22]:
!pip install streamlit transformers pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [26]:
%%writefile app.py
import streamlit as st
from transformers import pipeline

# ✅ Set page config FIRST
st.set_page_config(page_title="News Summarizer App", layout="wide")

# Load pipelines once
@st.cache_resource
def load_pipelines():
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    headline_gen = pipeline("summarization", model="google/pegasus-xsum")
    return summarizer, headline_gen

summarizer, headline_gen = load_pipelines()

# Helper functions
def summarize_article(text):
    chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
    summary = ""
    for chunk in chunks:
        out = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
        summary += out[0]['summary_text'] + " "
    return summary.strip()

def generate_headline(text):
    result = headline_gen(text, max_length=15, min_length=5, do_sample=False)
    return result[0]['summary_text']

def extract_subpoints(summary):
    sentences = summary.split('. ')
    return [f"• {s.strip()}." for s in sentences if s][:5]

# Streamlit UI
st.title("📰 News Summarizer & Headline Generator")

article = st.text_area("Paste a news article here:", height=300)

if st.button("Generate Summary"):
    if len(article.strip()) == 0:
        st.warning("Please paste a news article first.")
    else:
        with st.spinner("Processing..."):
            headline = generate_headline(article)
            summary = summarize_article(article)
            subpoints = extract_subpoints(summary)

        st.subheader("🗞️ Headline")
        st.success(headline)

        st.subheader("📌 Key Points")
        for pt in subpoints:
            st.markdown(pt)

        st.subheader("📝 Summary Paragraph")
        st.info(summary)


Overwriting app.py


In [28]:
from pyngrok import ngrok

ngrok.kill()
!streamlit run app.py &> /content/log.txt &
public_url = ngrok.connect(8501, "http")
print("🌐 Open your app here:", public_url)


🌐 Open your app here: NgrokTunnel: "https://6923-35-221-5-3.ngrok-free.app" -> "http://localhost:8501"


In [40]:
%%writefile app.py
import streamlit as st
from transformers import pipeline

# Set page config
st.set_page_config(page_title="📰 News Summarizer", layout="wide")

# Inject custom CSS for styling
st.markdown("""
<style>
    .headline-box {
        background: linear-gradient(90deg, #4b6cb7 0%, #182848 100%);
        padding: 20px;
        border-radius: 10px;
        color: white;
        font-size: 22px;
        font-weight: bold;
        margin-bottom: 20px;
    }
    .bullet-point {
        background-color: #f1f3f6;
        color: #000000;
        padding: 10px;
        border-radius: 8px;
        margin-bottom: 8px;
        font-size: 16px;
    }
    .summary-box {
        background-color: #e3f2fd;
        padding: 15px;
        border-radius: 8px;
        color: #333;
        font-size: 17px;
    }
    .section-header {
        font-size: 24px;
        margin-top: 30px;
        color: ##0000FF;
        font-weight: 700;
    }
</style>
""", unsafe_allow_html=True)

# Load NLP models once
@st.cache_resource
def load_pipelines():
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    headline_gen = pipeline("summarization", model="google/pegasus-xsum")
    return summarizer, headline_gen

summarizer, headline_gen = load_pipelines()

# Summarization logic
def summarize_article(text):
    chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
    summary = ""
    for chunk in chunks:
        out = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
        summary += out[0]['summary_text'] + " "
    return summary.strip()

def generate_headline(text):
    result = headline_gen(text, max_length=15, min_length=5, do_sample=False)
    return result[0]['summary_text']

def extract_subpoints(summary):
    sentences = summary.split('. ')
    return [s.strip() for s in sentences if s][:5]

# Sidebar info
with st.sidebar:
    st.image("https://img.icons8.com/color/96/news.png", width=80)
    st.markdown("## 📰 News Summarizer")
    st.markdown("""
    This app takes a news article and generates:
    - 🏷️ A headline
    - 📌 Bullet points
    - 📝 A summary paragraph
    """)
    st.markdown("---")
    st.caption("Made by Sahejpreet")

# App title
st.markdown("<h1 style='text-align: center; color: #4b6cb7;'>✨ News Summarizer & Headline Generator</h1>", unsafe_allow_html=True)
st.markdown("---")

# Text input
article = st.text_area("✍️ Paste your news article below:", height=300)
st.caption(f"📝 Word Count: {len(article.split())}")

# Generate button
if st.button("🚀 Generate Summary"):
    if len(article.split()) < 30:
        st.warning("Please enter at least 30 words.")
    else:
        with st.spinner("Analyzing and summarizing..."):
            headline = generate_headline(article)
            summary = summarize_article(article)
            subpoints = extract_subpoints(summary)

        # Headline
        st.markdown("<div class='section-header'>🗞️ Generated Headline</div>", unsafe_allow_html=True)
        st.markdown(f"<div class='headline-box'>{headline}</div>", unsafe_allow_html=True)

        # Bullet Points
        st.markdown("<div class='section-header'>📌 Key Points</div>", unsafe_allow_html=True)
        for pt in subpoints:
            st.markdown(f"<div class='bullet-point'>🔹 {pt}</div>", unsafe_allow_html=True)

        # Summary
        st.markdown("<div class='section-header'>📝 Summary Paragraph</div>", unsafe_allow_html=True)
        st.markdown(f"<div class='summary-box'>{summary}</div>", unsafe_allow_html=True)


Overwriting app.py


In [41]:
from pyngrok import ngrok

ngrok.kill()
!streamlit run app.py &> /content/log.txt &
public_url = ngrok.connect(8501, "http")
print("🌐 Open your app here:", public_url)


🌐 Open your app here: NgrokTunnel: "https://060e-35-221-5-3.ngrok-free.app" -> "http://localhost:8501"
